In [1]:
# !pip install autoviz
# !pip install evidently
# !pip install --pre pycaret
# !pip install streamlit

In [12]:
!pip install pycaret

  Using cached pycaret-2.3.10-py3-none-any.whl (320 kB)
  Using cached scikit-learn-0.23.2.tar.gz (7.2 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'error'


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [11]:
from autoviz.AutoViz_Class import AutoViz_Class
# from pandas_profiling import ProfileReport
from fastai.tabular.core import cont_cat_split
from yellowbrick.classifier import confusion_matrix
from yellowbrick.classifier import classification_report

from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split,\
                                    GridSearchCV
from sklearn.preprocessing import (StandardScaler, 
                                   OneHotEncoder)
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn import metrics

import pandas as pd
import numpy as np

from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

import plotly.express as px

# ML 03
from evidently.dashboard import Dashboard
from evidently.dashboard.tabs import DataDriftTab, CatTargetDriftTab

from pycaret.datasets import get_data
from pycaret import regression
from pycaret import arules

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
import pickle 

%matplotlib inline

ModuleNotFoundError: No module named 'pycaret'

# Propósito do modelo preditivo (need statement)

Este modelo tem a função de auxiliar nossos clientes na revenda de seus carros. 

A base de dados utilizada está disponível em: https://www.kaggle.com/datasets/adityadesai13/used-car-dataset-ford-and-mercedes?resource=download&select=ford.csv

O conjunto de dados contém informações de preço, transmissão, quilometragem, tipo de combustível, imposto rodoviário, milhas por galão (mpg) e tamanho do motor.



# Análise exploratória de dados (gráfico e tabela)

Nesta seção, seão apresentados alguns gráficos e tabelas a fim de explorar o conjunto de dados.

In [ ]:
data = pd.read_csv('ford.csv')
# data = pd.read_csv('/kaggle/input/used-car-dataset-ford-and-mercedes/ford.csv')

In [ ]:
# profile = ProfileReport(data, title="Pandas Profiling Report")
# profile.to_file("profile.html")

In [ ]:
data.describe() # algumas estatísticas básicas para os dados numéricos

In [ ]:
cont_names, cat_names = cont_cat_split(data)
for c in ['year','mileage','price']:
    fig = px.histogram(data[cont_names], x=c, marginal="box")
    fig.update_layout(title_text=f"Histogram for {c}")
    fig.show()
    fig = px.histogram(data[cont_names], x=c,color='year')
    fig.show()

In [ ]:
fig = px.histogram(data[cat_names], x='model', color="model")
fig.update_xaxes(type='category')
plt.savefig('barmodel.jpg')
fig.show()

In [ ]:
count_model = pd.DataFrame(data['model'].value_counts())
count_model

In [ ]:
text = " ".join(w for w in data['model'].values)
word_cloud = WordCloud(
        width=3000,
        height=2000,
        random_state=1,
        background_color="salmon",
        colormap="Pastel1",
        collocations=False,
        stopwords=STOPWORDS,
        ).generate(text)
plt.figure(figsize=(10,8))
plt.imshow(word_cloud)
plt.axis("off")
plt.savefig('wordcloud.jpg')
plt.show()

In [ ]:
top5 = count_model.head(5).index
plt.figure(figsize=(10,8))
# top5 carros e densidade dos preços
data.loc[data['model'].isin(top5)].groupby('model')['price'].plot(kind='density',legend=True)
plt.show()



# Engenharia de features e tratamento dos dados



In [ ]:
# AV = AutoViz_Class()
# dft = AV.AutoViz(filename="", dfte=data, chart_format='html')

In [ ]:
tail = count_model.tail(10).index
tail

In [ ]:
# combinando categorias raras
outros = len(tail)*['Outros']
dic = dict(zip(tail,outros))
data['model'] = data['model'].replace(dic,regex=True)
pd.DataFrame(data['model'].value_counts())

In [ ]:
# transformando os dados categóricos e normalizando os numéricos
y = data['price']
X = data.drop(columns=['price'])

# dados numericas e categoricas
cont_names, cat_names = cont_cat_split(X)

# separando em treino e teste (70/30)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# categorical_transformer
categorical_transformer = OneHotEncoder(handle_unknown="ignore")
dt = categorical_transformer.fit_transform(X_train[cat_names]).toarray()
cols = categorical_transformer.get_feature_names_out()
X_train_cat = pd.DataFrame(dt, columns=cols)
X_train_cat['fuelType_Electric'] = 0

dt = categorical_transformer.fit_transform(X_test[cat_names]).toarray()
cols = categorical_transformer.get_feature_names_out()
X_test_cat = pd.DataFrame(dt, columns=cols)
X_test_cat['fuelType_Other'] = 0
X_test_cat = X_test_cat.reindex(columns=X_train_cat.columns)

# numeric_transformer
scaler = StandardScaler().fit(X_train[cont_names])
X_train_num = pd.DataFrame(scaler.fit_transform(X_train[cont_names]), columns=cont_names)

scaler = StandardScaler().fit(X_test[cont_names])
X_test_num = pd.DataFrame(scaler.fit_transform(X_test[cont_names]), columns=cont_names)

X_train = pd.concat([X_train_num, X_train_cat], axis=1)
X_test = pd.concat([X_test_num, X_test_cat], axis=1)

In [ ]:
# # outra forma - transformando os dados categóricos e normalizando os numéricos
# y = data['price']
# X = data.drop(columns=['price'])

# # dados numericas e categoricas
# cont_names, cat_names = cont_cat_split(X)

# # separando em treino e teste (70/30)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# numeric_transformer = Pipeline(
#     steps=[("imputer", KNNImputer(n_neighbors=5)), ("scaler", StandardScaler())])

# categorical_transformer = Pipeline(
#     steps=[("ohe", OneHotEncoder(handle_unknown="ignore", drop='first'))])
    
# preprocessor = ColumnTransformer(transformers=[ 
#         ("num", numeric_transformer, cont_names),
#         ("cat", categorical_transformer, cat_names)])

# X_train = preprocessor.fit_transform(X_train) 
# X_test = preprocessor.fit_transform(X_test) 
# len(X_train), X_train



# Data drift (sugestão, utilizar o evidently)

In [ ]:
dashboard = Dashboard(tabs=[DataDriftTab(verbose_level=0)])
dashboard.calculate(X_train, X_test, column_mapping=None)
dashboard.show(mode='inline')

In [ ]:
y_train_target = pd.DataFrame(data=y_train).rename(columns={'price':'target'})
y_test_target = pd.DataFrame(data=y_test).rename(columns={'price':'target'})

dashboard = Dashboard(tabs=[CatTargetDriftTab(verbose_level=0)])
dashboard.calculate(y_train_target, y_test_target)
dashboard.show(mode='inline')



# Construção do modelo preditivo e principais métricas

De acordo com o Pandas Profiling Report, observa-se que o preço tem alta correlação com as seguintes características: 

$$'model',\, 'year',\, 'transmission',\, 'mileage',\, 'fuelType'$$

In [ ]:
X['model'].unique()

In [ ]:
# seleção de feature para criação do modelo preditivo
features, target = ['model', 'year', 'transmission', 'mileage', 'fuelType'],['price']

X = data[features]
y = data[target]

Xy = X.copy()
Xy['price'] = y
Xy

In [ ]:
# # apois para encontar melhor modelo preditivo para o conjunto de dados
# s = regression.setup(Xy, target = 'price', session_id=123)
# best = regression.compare_models()

In [ ]:
# gbr = regression.create_model('gbr')
# regression.plot_model(gbr, plot='residuals')
# regression.plot_model(gbr, plot='error')
# with open('ford.pickle', 'wb') as model:
#     pickle.dump(gbr, model)

In [ ]:
y = Xy['price']
X = Xy.drop(columns=['price'])

# dados numericas e categoricas
cont_names, cat_names = cont_cat_split(X)

# categorical_transformer
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

dt = categorical_transformer.fit_transform(X[cat_names]).toarray()
X_train_cat = pd.DataFrame(dt, columns=categorical_transformer.get_feature_names_out())
# X_train_cat['fuelType_Electric'] = 0

dt = categorical_transformer.fit_transform(X[cat_names]).toarray()
X_cat = pd.DataFrame(dt, columns=categorical_transformer.get_feature_names_out())
# X_cat['fuelType_Other'] = 0
# X_cat = X_cat.reindex(columns=X_train_cat.columns)

# numeric_transformer
scaler = StandardScaler().fit(X[cont_names])
X_num = pd.DataFrame(scaler.fit_transform(X[cont_names]), columns=cont_names)

scaler = StandardScaler().fit(X[cont_names])
X_num = pd.DataFrame(scaler.fit_transform(X[cont_names]), columns=cont_names)

X = pd.concat([X_num, X_cat], axis=1)

# separando em treino e teste (70/30)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
reg = LinearRegression()
reg.fit(X_train, y_train)

with open('lin_reg_ford.pickle', 'wb') as model:
    pickle.dump(reg, model)

In [ ]:
# !pip install matplotlib==3.1.3
n = 30
y_pred = reg.predict(X_test.iloc[:n])
x = range(1,n+1)
plt.plot(x,y_test.iloc[:n],label='test')
plt.plot(x,y_pred,label='train')
plt.title(f'Preço dos {n} primeiros carros de teste')
if len(x) <= 20:
    plt.xticks(x, x)
plt.legend()
plt.show()

In [ ]:
X.shape

In [ ]:
cX = X.columns
len(cX),cX

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import pickle

st.set_page_config(
    page_title='Ford Preços',
    page_icon=":car:"
)

with open('lin_reg_ford.pickle', 'rb') as model:
    reg = pickle.load(model)

cols = ['year', 'mileage', 'model_ B-MAX', 'model_ C-MAX', 'model_ EcoSport',
        'model_ Edge', 'model_ Fiesta', 'model_ Focus', 'model_ Galaxy',
        'model_ Grand C-MAX', 'model_ KA', 'model_ Ka+', 'model_ Kuga',
        'model_ Mondeo', 'model_ S-MAX', 'model_Outros',
        'transmission_Automatic', 'transmission_Manual',
        'transmission_Semi-Auto', 'fuelType_Diesel', 'fuelType_Electric',
        'fuelType_Hybrid', 'fuelType_Other', 'fuelType_Petrol']

car_user = pd.DataFrame(np.zeros((1,24)),index=[0],columns=cols)
    
model = st.selectbox('Qual modelo do seu carro?', (None, ' Fiesta', ' Focus', 'Outros', ' Kuga', ' EcoSport', ' C-MAX',\
                                                   ' Mondeo', ' Ka+', ' S-MAX', ' B-MAX', ' Edge', ' Grand C-MAX',' KA', ' Galaxy'))

transmission = st.selectbox('Qual o tipo de transmissão do seu carro?', (None, 'Automatic', 'Manual', 'Semi-Auto'))

fuelType = st.selectbox('Qual o tipo de combustível do seu carro?', (None, 'Petrol', 'Diesel', 'Hybrid', 'Electric', 'Other'))

year = st.number_input('Qual o ano do seu carro?', min_value=2000, max_value=2022, value=2022, step=1)

mileage = st.number_input('Qual a quilmetragem do seu carro?', min_value=0, max_value=None, value=0, step=None)


char_index = ['Modelo', 'Transmissão', 'Combustível', 'Ano', 'Milhas']
char = [model, transmission, fuelType, year, mileage]

if None in char:
    st.write('Preciso de todas as características solicitadas para tentar acertar qual é o seu animal')
else:
    car_user['model_'+ model] = 1
    car_user['transmission_' + transmission] = 1
    car_user['fuelType_'+ fuelType] = 1
    car_user['year'] = year
    car_user['mileage'] = mileage
    
    df = pd.DataFrame(car_user, index = [0])
    # st.dataframe(df)
    df = pd.DataFrame(char, index =char_index )
    st.dataframe(df.transpose())
    
    price_pred = reg.predict(np.array(car_user).reshape(1,-1))
    
    st.write(f'Sugerimos o preço de venda para o seu carro sendo {price_pred}')                 


In [ ]:
for lab in ['model','year', 'transmission', 'mileage', 'fuelType']:
    print(data[lab].unique())



# Principais métricas